There are several ways to cross-match an external catalog with Gaia DR2 sources.

## Using web interface

First, while it is possible to upload a catalog for querying anonymously, the web interface at the official Gaia Archive does not seem to provide this functionality. Only a list of target 'names' can be provided with the current interface (see [Using anynomous upload](#Using-anynomous-upload)).

If you still prefer using the web interface to scripting:

1. create an account, which grants you 1GB of user tables quota, and login.
2. Go to Search > ADQL Tab and upload your table. It can be provided in VOTable, CSV or ASCII. Once it is uploaded, you will see it under "User Tables" as `user_[username].[tablename]`.
3. You can now use this table just like any other table in the database. For example, to cross-match `user_joe.mytable` with `gaiadr2.gaia_source`:

```sql
SELECT t1.ra AS "t1_ra", t1.dec AS "t1_dec", t2.ra AS "gaia_ra", t2.dec AS "gaia_dec", ...
FROM user_joe.mytable AS t1, gaiadr2.gaia_source AS t2
WHERE 1 = CONTAINS(POINT('ICRS', t1.ra, t1.dec), CIRCLE('ICRS', t2.ra, t2.dec));
```

- **NOTE**: The uploaded user tables can be edited. Check box of a table and click 'Edit table'. Specifically, you can assign RA and Dec columns, which will make those columns indexed. I suppose this may help the cross-match to be faster but I am not sure if it matters much with smaller tables.

- **NOTE**: There is also 'Cross-match tables' button, which does this query for you.

- **NOTE**: It is much faster to do
```sql
1 = CONTAINS(POINT(smaller_catalog), CIRCLE(larger_catalog))
```
than the other way around.

- **NOTE**: If you are planning to combine the resulting table with other tables, it can be useful to have some internal index to check unique/non-existent entries as well as merging tables. You can also do a `LEFT JOIN` instead:

```sql
SELECT t1.ra AS "t1_ra", t1.dec AS "t1_dec", t2.ra AS "gaia_ra", t2.dec AS "gaia_dec", ...
FROM user_joe.mytable AS t1
LEFT JOIN gaiadr2.gaia_source AS t2
  ON 1 = CONTAINS(POINT('ICRS', t1.ra, t1.dec), CIRCLE('ICRS', t2.ra, t2.dec));
```

This way, the result will containt matches for every row in t1:
- it will contain t1 rows without any matches
- it can contain duplicated t1 rows if there are multiple matches to a single row in t1.


## Using astroquery.gaia

### Using user tables - NOT AVAILABLE YET!

(This feature is not available yet but it probably will be soon as there's work being done. Ignore for now.)

The same action above, uploading a table to user space and using it to run queries, can be done in python with `astroquery.gaia` module. 

```python
from astroquery.gaia import Gaia

Gaia.login()   # will ask username and password

Gaia.upload_user_table(...)
```

### Using anonymous upload

You do not have to upload a table to user space with an account. You can use on-the-fly uploaded tables in your synchronous or asynchronous cross-matches (This is the functionality that the web interface seems to be missing).

See https://astroquery.readthedocs.io/en/latest/gaia/gaia.html for details.

Note that anonymous uploads fall under `TAP_UPLOAD` schema instead of `user_[username]`.

## Using CDS xMatch

It is also possible to use [CDS xmatch service](http://cdsxmatch.u-strasbg.fr/#tab=xmatch&) as CDS also hosts the entire Gaia DR2 data. This can be particularly useful if you want to use other tables from hosted at CDS like machine-readable tables published in journal articles.

Astroquery has a python interface to the service: https://astroquery.readthedocs.io/en/latest/xmatch/xmatch.html.
